In [ ]:
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip -O /tmp/rps.zip
!unzip -d data/ /tmp/rps.zip 

In [2]:
import os
import shutil
import tensorflow as tf

from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
rock_train, rock_test = train_test_split(os.listdir("/content/data/rps/rock"), 
                                         test_size = 0.2, random_state = 123)
paper_train, paper_test = train_test_split(os.listdir("/content/data/rps/paper"), 
                                           test_size = 0.2, random_state = 123)
scissors_train, scissors_test = train_test_split(os.listdir("/content/data/rps/scissors"), 
                                                 test_size = 0.2, random_state = 123)

In [4]:
def move_images(src_folder, dst_folder, filenames):
    for file in filenames:
        src = os.path.join(src_folder, file)
        dst = os.path.join(dst_folder, file)
        if not os.path.exists(dst): shutil.move(src, dst)

# Create directories for training and test data
if not os.path.exists("/content/data/train"):
    os.mkdir("/content/data/train")
    os.mkdir("/content/data/train/rock")
    os.mkdir("/content/data/train/paper")
    os.mkdir("/content/data/train/scissors")

if not os.path.exists("/content/data/test"):
    os.mkdir("/content/data/test")
    os.mkdir("/content/data/test/rock")
    os.mkdir("/content/data/test/paper")
    os.mkdir("/content/data/test/scissors")

# Move images then delete source folders
if os.path.exists("/content/data/rps"):
    move_images("/content/data/rps/rock", "/content/data/train/rock", rock_train)
    move_images("/content/data/rps/paper", "/content/data/train/paper", paper_train)
    move_images("/content/data/rps/scissors", "/content/data/train/scissors", scissors_train)
    move_images("/content/data/rps/rock", "/content/data/test/rock", rock_test)
    move_images("/content/data/rps/paper", "/content/data/test/paper", paper_test)
    move_images("/content/data/rps/scissors", "/content/data/test/scissors", scissors_test)

    for folder in os.listdir("/content/data/rps"):
        os.removedirs(os.path.join("/content/data/rps", folder))

In [5]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.35,
    fill_mode = 'nearest',
)

val_datagen = ImageDataGenerator(rescale = 1./225)

In [6]:
train_generator = train_datagen.flow_from_directory(
    "/content/data/train",
    target_size=(150,150),
    batch_size= 32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    "/content/data/test",
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.


In [7]:
# Define CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = "relu", input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(3, activation = "softmax")   
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [8]:
# configure model and callbacks for training 
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss="categorical_crossentropy",
    metrics = ['accuracy']
)

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.97):
            self.model.stop_training = True

callbacks = [
    myCallback(),
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5)
]

In [9]:
# start training with callbacks
model.fit(
    train_generator, 
    validation_data = val_generator,
    epochs=50, 
    steps_per_epoch = 63,
    validation_steps = 16,
    verbose = 1, 
    callbacks = callbacks
)

Epoch 1/50
63/63 [==============================] - 31s 280ms/step - loss: 0.7694 - accuracy: 0.6746 - val_loss: 0.1480 - val_accuracy: 0.9504 - lr: 5.0000e-04
Epoch 2/50
63/63 [==============================] - 17s 266ms/step - loss: 0.1485 - accuracy: 0.9544 - val_loss: 0.0584 - val_accuracy: 0.9861 - lr: 5.0000e-04
Epoch 3/50
63/63 [==============================] - 17s 262ms/step - loss: 0.1033 - accuracy: 0.9683 - val_loss: 0.0458 - val_accuracy: 0.9881 - lr: 5.0000e-04
Epoch 4/50
63/63 [==============================] - 18s 287ms/step - loss: 0.0659 - accuracy: 0.9851 - val_loss: 0.0277 - val_accuracy: 0.9921 - lr: 5.0000e-04
